In [ ]:
from tools import *
from omegaconf import OmegaConf
from pprint import PrettyPrinter
from models._main import get_model
from models._utils import print_parameters

sns.set_style("whitegrid")
pp = PrettyPrinter(indent=4, width=100, compact=True)

# Color and Marker code

*Returns the legend code that is used across all plots.*

In [ ]:
leg_code = legend_code()
pp.pprint(leg_code)

# Model Parameters

In [ ]:
model_config = OmegaConf.load("../../configs/submodule/model.yaml")

model_config.model.use_this_pretrained_model = None
model = get_model(model_config.model)
print_parameters(model, verbose=True)

# Dataset information

*Plot information from datasets. I'm loading information from datasets that used all available worms and neurons in the analysis (in this case, the data scaling experiment datasets, that used the maximum amount of data)*

In [ ]:
fig1_path_dict = {
    "train_dataset_info": "/om2/user/qsimeon/worm-graph/data/custom_train/train_dataset_info.csv",  # Path to train dataset info => extract number of train time steps
    "val_dataset_info": "/om2/user/qsimeon/worm-graph/data/custom_validation/val_dataset_info.csv",  # Path to val. dataset info => extract number of val. time steps
    "combined_dataset_info": "/om2/user/qsimeon/worm-graph/data/custom_combined/combined_dataset_info.csv",  # Path to combined dataset info => extract total number of worms and time step interval
}

# NOTE: Error bars on bar plots show +/- 2 SEM
dataset_info = dataset_information(path_dict=fig1_path_dict, legend_code=leg_code)

# Data scaling

*Data scaling plot. To generate the dataframe with the relevant results, firstly for each model you want to plot, you need to furnish the paths for each trial. Then you can call the plot function to obtain a graphic with mean and standard deviation bars. You need to provide the legend code for color standardization.*

In [ ]:
# TODO: Do experiments, for each model architecture, by varying the sizes of subsets of the combined datasets.
# Each subset will have a distribution of timesteps per neuron (i.e. each worm has a timestep per neuron metric).
# For each subset, repeat the train pipeline n times (here you did n=3) and save the validation loss each time.
nts_experiments = {
    "LSTM": [
        os.path.join("results", "NTS_LSTM"),  # LSTM trial 1
        os.path.join("results", "NTS_LSTM_1"),  # LSTM trial 2
        os.path.join("results", "NTS_LSTM_2"),  # LSTM trial 3
    ],
    "Transformer": [
        os.path.join("results", "NTS_TRANSFORMER"),  # Transformer trial 1
        os.path.join("results", "NTS_TRANSFORMER_1"),  # Transformer trial 2
        os.path.join("results", "NTS_TRANSFORMER_2"),  # Transformer trial 3
    ],
    "Linear": [
        os.path.join("results", "NTS_LINEAR"),  # Linear trial 1
        os.path.join("results", "NTS_LINEAR_1"),  # Linear trial 2
        os.path.join("results", "NTS_LINEAR_2"),  # Linear trial 3
    ],
}

data_results = data_scaling_df(
    nts_experiments
)  # Retrieve the relevant results for the data scaling (and hidden scaling) plot
data_results.head(2)  # Took ~1s in my computer (small experiments)

In [ ]:
data_scaling_plot(data_results, legend_code=leg_code)

# Hidden Scaling

*Hidden dimension scaling plot. To generate the dataframe with the relevant results, firstly for each model you want to plot, you need to furnish the paths for each trial (it can take some minutes). Then you can call the plot function to obtain a graphic with mean and standard deviation bars. You can chose the degree of the polynomial fit used. You need to provide the legend code for color standardization.*

In [ ]:
# Instead of plotting against the hidden size, plot against the number of parameters.
# The hidden size is still the variable that we will vary, but it is simply a way to change the number of parameters.

hidden_experiments = {
    "LSTM": [
        os.path.join("results", "HIDDEN_LSTM_1"),  # LSTM trial 1
        os.path.join("results", "HIDDEN_LSTM_2"),  # LSTM trial 2
        os.path.join("results", "HIDDEN_LSTM_3"),  # LSTM trial 3
    ],
    "Transformer": [
        os.path.join("results", "HIDDEN_TRANSFORMER_1"),  # Transformer trial 1
        os.path.join("results", "HIDDEN_TRANSFORMER_2"),  # Transformer trial 2
        os.path.join("results", "HIDDEN_TRANSFORMER_3"),  # Transformer trial 3
    ],
    "Linear": [
        os.path.join("results", "HIDDEN_LINEAR_1"),  # Linear trial 1
        os.path.join("results", "HIDDEN_LINEAR_2"),  # Linear trial 2
        os.path.join("results", "HIDDEN_LINEAR_3"),  # Linear trial 3
    ],
}

hidden_results = data_scaling_df(
    hidden_experiments
)  # Retrieve the relevant results for the hidden scaling (and data scaling) plot
hidden_results.head(2)  # Took ~4min in my computer (big experiments)

In [ ]:
hidden_scaling_plot(hidden_results, legend_code=leg_code, fit_deg=2)

# Data scaling slopes

*Data scaling slopes plot. To generate the dataframe with the relevant results, firstly for each model you want to plot, you need to furnish the paths for each trial (it can take some minutes). Then you can call the plot function to obtain a graphic with mean and standard deviation bars. You need to provide the legend code for color standardization.*

*To create this plot, after a model has been trained (with a given amount of data), we validate it on individual experimental datasets. We then make a plot for each model by experimental dataset.*

In [ ]:
nts_experiments = {
    "LSTM": [
        os.path.join("results", "NTS_LSTM"),  # LSTM trial 1
        os.path.join("results", "NTS_LSTM_1"),  # LSTM trial 2
        os.path.join("results", "NTS_LSTM_2"),  # LSTM trial 3
    ],
    "Transformer": [
        os.path.join("results", "NTS_TRANSFORMER"),  # Transformer trial 1
        os.path.join("results", "NTS_TRANSFORMER_1"),  # Transformer trial 2
        os.path.join("results", "NTS_TRANSFORMER_2"),  # Transformer trial 3
    ],
    "Linear": [
        os.path.join("results", "NTS_LINEAR"),  # Linear trial 1
        os.path.join("results", "NTS_LINEAR_1"),  # Linear trial 2
        os.path.join("results", "NTS_LINEAR_2"),  # Linear trial 3
    ],
}

scaling_slope_results = scaling_slopes_df(nts_experiments)  # Took ~10s
scaling_slope_results.head(2)

In [ ]:
scaling_slopes_plot(scaling_slope_results, legend_code=leg_code)

# Cross-dataset Generalization

*This function automatically loads the results from the indicated experiments and displays a heatmap plot showing the validation loss on individual experimental datasets after a model has been trained (from scratch) using a single experimental dataset.*

*You need to specify the model names respective to each experiment and also the legend color code.*

In [ ]:
experiment_log_folders = [
    os.path.join("results", "CDS_LSTM"),  # LSTM trial 1
    os.path.join("results", "CDS_TRANSFORMER"),  # Transformer trial 1
    os.path.join("results", "CDS_LINEAR"),  # Linear trial 1
]

model_names = ["LSTM", "Transformer", "Linear"]

cross_dataset(
    experiment_log_folders=experiment_log_folders,
    model_names=model_names,
    legend_code=leg_code,
)

# Predictions

*Load and plot the predictions for the same neuron across different datasets. You need to determine which neurons are common to all datasets (I did it manually).*

In [ ]:
experiment_log_folders = [
    os.path.join("results", "NTS_LSTM_2"),  # LSTM trial 1
    os.path.join("results", "NTS_TRANSFORMER_2"),  # Transformer trial 1
    os.path.join("results", "NTS_LINEAR_2"),  # Linear trial 1
]

model_names = ["LSTM", "Transformer", "Linear"]

ds_type = "val"  # we are plotting the predictions using the validation dataset (data never seen during training, second split)
exp = "exp5"  # model trained with maximum amount of data (best model to use for predictions)
neuron_to_plot = "AVER"  # neuron we want to plot

teacher_forcing(
    experiment_log_folders=experiment_log_folders,
    model_names=model_names,
    legend_code=leg_code,
    ds_type=ds_type,
    exp=exp,
    neuron_to_plot=neuron_to_plot,
)

In [ ]:
autoregressive(
    experiment_log_folders=experiment_log_folders,
    model_names=model_names,
    legend_code=leg_code,
    ds_type=ds_type,
    exp=exp,
    neuron_to_plot=neuron_to_plot,
)

# (Sup. fig) Prediction gap

*Loads the relevant results and plots the refinement of the prediction plot as the amount of training data for the LSTM model increases.*

In [ ]:
pred_gap_df = prediction_gap(
    exp_nts_log_dir=os.path.join("results", "NTS_LSTM_1"),
    legend_code=leg_code,
    neuronID="AVER",
    wormID="worm1",
    datasetID="Flavell2023",
)